In [ ]:
! pip install --upgrade --user google-cloud-aiplatform ragstack-ai

In [ ]:
import getpass, os, requests, sys
import vertexai, json
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part
)
from astrapy.db import AstraDB, AstraDBCollection
from google.colab import files
from langchain_google_vertexai import ChatVertexAI
from langchain.schema.messages import HumanMessage
from PIL import Image, ImageFile
import pandas as pd


In [ ]:
PROJECT_ID = 'red-delight-346705'
LOCATION = 'us-central1'
ASTRA_DB_API_ENDPOINT = 'ASTRA_DB_API_ENDPOINT'
ASTRA_DB_APPLICTAION = 'ASTRA_DB_APPLICATION'

In [ ]:
if "GCP_PROJECT_ID" not in os.environ:
    os.environ["GCP_PROJECT_ID"] = getpass.getpass("Provide your GCP Projcet ID")

if "LOCATION" not in os.environ:
    os.environ["LOCATION"] = getpass.getpass("Provide your GCP Location")

if "ASTRA_DB_ENDPOINT" not in os.environ:
    os.environ["ASTRA_DB_ENDPOINT"] = getpass.getpass("Provide your Astra DB Endpoint")



These three cells are for Colab only

In [ ]:
!gcloud confing set project {os.getenv("GCP_PROJECT_ID")}

In [ ]:
# Additional Authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
! gcloud auth list

Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
# Define project information
PROJECT_ID = os.getenv("GCP_PROJECT_ID")
LOCATION = os.getenv("LOCATION")

In [ ]:
# Initialize Vertex AI

vertexai.init(project=PROJECT_ID, location = LOCATION)

Use Gemini 1.0 Pro Model

In [ ]:
model =GenerativeModel("gemini-1.0-pro")

Generate text from text prompts

In [ ]:
responses = model.generate_content("why is the sky blue?", stream=True)

for response in responses:
    print(response.text, end="")

In [ ]:
prompt = """

    Create a numbered list of 10 items. Each item in the list should be trend in the tech industry.
    Each trend should be less than 5 words.

"""

In [ ]:
responses_1 = model.generate_content(prompt, stream=True)

In [ ]:
for response in responses_1:
    print(response.text, end="")

Model parameters

In [ ]:
generation_confing = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

In [ ]:
responses_2 = model.generate_content(
    "Why is the sky blue?",
    generation_confing=generation_confing,
    stream=True,
)

In [ ]:
for response in responses_2:
    print(response.text, end="")

In [ ]:
source_img_data = requests.get("image_of_coffee_maker")

In [ ]:
with open("coffee_maker_parts.png", "wb") as handler:
    handler.write(source_img_data)

In [ ]:
chat = ChatVertexAI(model_name="gemini-1.0-pro-vision", safety_settings={
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
},)

In [ ]:
image_message = {
    "type": "image_url",
    "image_url": {"url":"coffee_maker_part.png"},
}

text_message = {
    "type":"text",
    "text":"What is this image? Share a link to purchase a replacement"
}

In [ ]:
message = HumanMessage(content=[text_message, image_message])

output = chat([message])

print(output.content)

In [ ]:
d = {'name': ["Saucer", "Saucer Ceramic", "Milk Jug Assembly", "Handle Steam Wand Kit (New Version From 0735 PDC)", "Spout Juice Small (From 0637 tp 1041 PDC)", "Cleaning Steam Wand", "Jug Frothing", "Spoon Tamping 50mm", "Collar Grouphead", "and so on"],
'url': ["https://www.breville.com/us/en/parts-accessories/parts/sp0014946.html?sku=SP0014946", "https://www.breville.com/us/en/parts-accessories/parts/sp0014914"],
'price': ["10.95", "4.99", "14.95", "8.95", "10.95", "6.95", "24.95", "8.95", "6.95", "12.95", "12.95", "14.95", "10.95", "16.95", "11.95"],
'image': ["https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014946/tile.jpg", "and so on"]}

df = pd.DataFrame(data=d)
df

In [ ]:
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image

In [ ]:
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [ ]:
# Initialize out vector db
astra_db = AstraDB(token=os.getenv("ASTRA_DB_TOKEN"), api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"))

In [ ]:
collection = astra_db.create_collection(collection_name="coffee_shop_ecommerce", dimention=1408)

In [ ]:
for i in range(len(df)):
    name = df.loc[i, "name"]
    image = df.loc[i, "image"]
    price = df.loc[i, "price"]
    url = df.loc[i, "url"]

    # Download this product's image and save it to the Colab filesystem
    # In production system this binary data would be stored in Google Cloud Storage
    img_data = requests.get(image).content
    with open(f"{name}.png", "wb") as handler:
        handler.write(img_data)

    # load the image from filesystem and compute the embedding value
    img = Image.load_from_file(f"{name}.png")
    embeddings = model.get_embeddings(image=img, contextual_text=name)

    try:
        # Add to AstraDB Vector Database
        collection.insert_one({
            "_id":i,
            "name":name,
            "image":image,
            "url":url,
            "price":price,
            "$vector":embeddings.image_embedding,
        })

    except Exception as error:
        # if you've already added this record, skip the error message
        error_info = json.loads(str(error))
        if error_info[0]["errorCode"] == "DOCUMENT_ALREADY_EXISTS":
            print("Document already exists in the database. Skipping.")

In [ ]:
# Embed the similar item 
img = Image.load_from_file("coffee_maker_part.png")

In [ ]:
embeddings_1 = model.get_embeddings(image=img, contextual_text="A espresso machine part")

In [ ]:
embeddings_1

In [ ]:
# Perform the vector search against AstraDB Vector
documents = collection.vector_find(
    embeddings_1.image_embedding,
    limit=3,
)

In [ ]:
related_products_csv = "name, image, price, url\n"
for doc in documents:
    related_products_csv += f"{doc['name']}, {doc['image']}, {doc['price']}, {doc['url']},\n"

In [ ]:
print(related_products_csv)

In [ ]:
image_message_1 = {
    "type":"image_url",
    "image_url":{"url":"/content/coffee_part.png"},
}

text_message_1 = {
    "type":"text",
    "text":f"What we have in this image? share the URL and price to purchase a replacement. Here are related products {related_products_csv}"
}

In [ ]:
message_1 = HumanMessage(content=[text_message_1, image_message_1])

In [ ]:
chat_1 = ChatVertexAI(model_name="gemini-1.0-pro-vision", safety_settings={
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
},)

In [ ]:
output_1 = chat_1([message_1])

In [ ]:
print(output_1)